In [1]:
### 모듈 로딩
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch import optim
import torch.optim.lr_scheduler as lr_scheduler

from torchinfo import summary

from konlpy.tag import *

from collections import Counter
from custom import *

import os
import re
import pickle

import pandas as pd
import json
import numpy as np

In [2]:
komoran = Komoran()

In [3]:
# 불용어 리스트 생성
def stopwords():
    stop_path = './stopwords.txt'

    with open(stop_path, 'r', encoding='utf-8') as f:
        wordlist = f.readlines()

    stopwords = []
    for word in wordlist:
        stopwords.append(word.replace('\n',''))
    
    return stopwords

In [4]:
def tokenize(data):
    # 토큰화
    data = re.sub(r'[^ㄱ-힣\s]', '', data)
    data = re.sub(r'\s', ' ', data)
    tokens = [token for token in komoran.morphs(data) if token not in stopwords]

    # 단어 사전 불러오기
    with open('vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
    
    # 정수 인코딩 및 패딩
    unk_id = vocab['<UNK>']
    word_ids = [[vocab.get(token, unk_id)for token in text] for text in tokens]

    max_length = 200
    pad_id = vocab['<PAD>']
    word_ids = pad_sequences(word_ids, max_length, pad_id)

    return word_ids

#### 전처리

In [5]:
# 불용어 리스트
stopwords = stopwords()

In [6]:
data = '2014년 설립된 통신용 커넥터 전문기업인 위드웨이브. 이 회사는 삼성전자와 함께 5G용 초고주파용 커넥트 기술을 공동 개발 중이다.\n초고주파용 커넥터는 초고속 신호 전송회로의 핵심 부품, 그러나 현재 전량 미국·일본으로부터 수입하는 실정이다.\n때문에 위드웨이브는 기술 개발을 추진했지만 여의치 않았다.\n기술력이 있어도 연구개발 비용을 감당키 어려웠던 까닭이다.\n이 같은 사정을 알게 된 삼성전자의 지원 덕분에 5G용 초고주파용 커넥트 국산화의 길이 열렸다.\n삼성전자와 중소기업 간 상생협력이 한층 깊어질 전망이다.\n중소벤처기업부와 함께 기금을 조성하고 내년부터 2026년까지 5년간 지원한다.\n이 기금은 중소기업의 기술 개발, 소재·부품·장비(소부장) 국산화 등 국내 기술 생태계 조성에 쓰인다. 1일 삼성전자는 중기부와 공동투자형 기술개발 투자기금 조성 업무협약을 체결했다.\n공동투자형 기술 개발은 대기업·공공기관 등 투자기업들이 중기부와 공동으로 중소기업에 자금을 지원하면, 해당 중소기업은 투자기업이 필요로 하는 기술이나 제품을 개발하는 방식이다.\n협약에 따라 삼성전자와 중기부는 각 150억원씩 출연해 총 300억원의 기금을 마련한다.\n이 기금은 위드웨이브처럼 자금 조달이 어려운 중소기업들의 기술 경쟁력을 높이는 데 쓰알 예정이다.\n삼성전자는 지난 2013년 중기부와 협약을 맺고 올해까지 친환경, 신소재, 설비·핵심 부품 국산화 관련 선행 기술을 개발하는 31개 중소기업을 100억원을 지원했다.\n자금 외에도 과제 목표를 함께 설정하는 한편, 기술지도와 테스트, 피드백 등 기술개발을 위한 종합적인 지원을 제공했다.\n삼성전자는 중소기업에 대한 지원이 양사가 윈윈하는 모델이 됐다고 판단하고 있다.\n31개 과제 중 19개가 개발이 완료돼 마이크로 LED TV, 삼성페이 등 제품과 서비스를 개선시키는 데 기여했다.\n특히 기술 국산화를 통한 원가 절감, 선행 기술 확보 등 성과를 거뒀다.\n중소기업 역시 삼성전자와의 거래를 통해 신규 매출을 창출하고, 관련 특허와 기술력을 확보함으로써 지속 성장의 기반을 마련했다.\n이에 삼성전자는 중기부와 협약을 연장하고 기금을 새롭게 조성키로 했다.\n범위도 소부장 국산화에서 시스템반도체와 AI(인공지능)·바이오헬스·로봇 등 차세대 기술로 넓혔다.\n해당 분야는 삼성전자가 미래 먹거리로 점찍은 분야다.\n향후 다른 방식으로 중소기업과의 협력이 강화될 가능성도 점쳐진다.\n삼성전자가 최근 중소기업과의 상생협력을 강화하고 있다.\n2009년부터 우수기술 설명회를 개최해 중소기업이 적기에 기술을 확보할 수 있도록 지원 중이다.\n2015년부터는 보유 특허 2만7000건을 개방해 현재까지 1600여건이 중소기업에 무상으로 양도됐다.\n2018년부터는 자체 추진해오던 스마트공장 사업을 중기부와 협력, 종합지원 활동으로 발전시켰다.\n지원 대상이 된 공장에 사내 전문가를 파견해 원포인트 레슨식의 컨설팅을 진행하고 기술 관련 정보를 제공해준다.\n현재 삼성전자의 투자는 소부장 국산화에 마중물이 되고 있다.\n지난해 11월 삼성전자는 같은 날 반도체 세라믹 부품을 만드는 미코세라믹스(216억원), 반도체 장비가 주력인 뉴파워프라즈마(127억원), 반도체용 건식진공펌프를 만드는 엘오티베큠(189억원), 반도체전공정 장비 생산업체인 케이씨텍(207억원) 등 4개 반도체 소부장 기업에 741억원을 투자했다.\n그에 앞서 7월엔 반도체 노광공정 핵심 소재인 블랭크마스크 개발 업체인 에스앤에스텍에 659억원, 반도체 검사장비 제조업체인 와이아이케이에 473억원을 투자했다.\n이렇게 수혈된 자금으로 소부장 기술 국산화가 속도를 올리고 있다.\n일례로 엘오티베큠과 미코세라믹스가 개발하는 장비들은 일본 의존도가 70% 이상이다.\n때문에 이들 기업의 기술∙제조 경쟁력은 국내 산업계의 경쟁력 강화하는 선순환으로 이어진다는 지적이다.\n특히 이재용 부회장은 사회와의 동행을 강조하면서 중소기업과의 상생에 무게를 실고 있다.\n이 부회장이 그리는 뉴삼성의 핵심은 과거 공공연히 이뤄졌던 관행과의 단절에서 시작되기 때문이다.\n더욱이 혁신의 속도를 올리고 있는 상황에서 잠재력을 지닌 중소기업과의 협력은 삼성의 변화를 촉진시킬 수 있다고 판단하고 있다.\n재계 관계자는 데일리임팩트에 \\\"이 부회장이 선대와는 다른 방식, 국민의 보편적 정서에 맞춰 사회에 기여하겠다는 의지가 강하다\\\"면서 \\\"상생협력 모델이 더욱 정교하게 발전될 것으로 보인다\\\"고 밝혔다.'

In [7]:
# 토큰화, 인코딩, 패딩 수행
word_ids = tokenize(data)

In [8]:
print(word_ids)

[[   1 3707    0 ...    0    0    0]
 [ 897    1    0 ...    0    0    0]
 [ 113    0    0 ...    0    0    0]
 ...
 [   1    1    0 ...    0    0    0]
 [   1    1    1 ...    0    0    0]
 [3180    1    0 ...    0    0    0]]


In [41]:
# 데이터로더 적용
batch_size=32
wordTS = torch.tensor(word_ids).long()

#### 모델 로딩

In [42]:
# 저장 경로
SAVE_PATH = './models/'

# 모델 구조 및 파라미터 저장 파일명
SAVE_MODEL = SAVE_PATH+'loss_0.484_score0.788.pth'

In [43]:
newsModel = torch.load(SAVE_MODEL, weights_only=False)

In [44]:
summary(newsModel)

Layer (type:depth-idx)                   Param #
SentenceClassifier                       --
├─Embedding: 1-1                         2,560,256
├─LSTM: 1-2                              198,656
├─Linear: 1-3                            129
├─Dropout: 1-4                           --
Total params: 2,759,041
Trainable params: 2,759,041
Non-trainable params: 0

In [45]:
newsModel.eval()

with torch.no_grad():
    pre_val = newsModel(wordTS)
    result = torch.sigmoid(pre_val).item()

    # 결과 출력
    if result > 0.5:
        print('본문의 도메인 일관성 부족')
    else:
        print('좋은 기사')

RuntimeError: a Tensor with 460 elements cannot be converted to Scalar

In [13]:
pre_val

tensor([[-2.0711],
        [-2.0712],
        [-2.0715],
        [-2.0699],
        [-2.0708],
        [-2.0699],
        [-2.0708],
        [-2.0705],
        [-2.0708],
        [-2.0706],
        [-2.0715],
        [-2.0708],
        [-2.0708],
        [-2.0715],
        [-2.0699],
        [-2.0711],
        [-2.0708],
        [-2.0713],
        [-2.0708],
        [-2.0708],
        [-2.0715],
        [-2.0699],
        [-2.0708],
        [-2.0708],
        [-2.0712],
        [-2.0708],
        [-2.0706],
        [-2.0706],
        [-2.0709],
        [-2.0709],
        [-2.0712],
        [-2.0708],
        [-2.0613],
        [-2.0711],
        [-2.0708],
        [-2.0708],
        [-2.0705],
        [-2.0711],
        [-2.0713],
        [-2.0708],
        [-2.0708],
        [-2.0710],
        [-2.0703],
        [-2.0708],
        [-1.0955],
        [-2.0710],
        [-2.0708],
        [-2.0708],
        [-2.0711],
        [-2.0708],
        [-2.0706],
        [-2.0700],
        [-2.